In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV
from LLS_main import LLS

In [3]:
data  = pd.read_csv("HousePrice.csv")
data.head()

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1.850000e+09,61666.67
1,60,1,True,True,True,Shahran,1.850000e+09,61666.67
2,79,2,True,True,True,Pardis,5.500000e+08,18333.33
3,95,2,True,True,True,Shahrake Qods,9.025000e+08,30083.33
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09,233333.33


In [4]:
data["Price(USD)_2023"] = data["Price(USD)"].multiply(1.633)
data["Price"] = data["Price"].map('{:,.0f}'.format)

data

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD),Price(USD)_2023
0,63,1,True,True,True,Shahran,"1,850,000,000",61666.67,100701.67211
1,60,1,True,True,True,Shahran,"1,850,000,000",61666.67,100701.67211
2,79,2,True,True,True,Pardis,"550,000,000",18333.33,29938.32789
3,95,2,True,True,True,Shahrake Qods,"902,500,000",30083.33,49126.07789
4,123,2,True,True,True,Shahrake Gharb,"7,000,000,000",233333.33,381033.32789
...,...,...,...,...,...,...,...,...,...
3260,63,1,True,True,False,Feiz Garden,"1,890,000,000",63000.00,102879.00000
3261,86,2,True,True,True,Southern Janatabad,"3,500,000,000",116666.67,190516.67211
3262,83,2,True,True,True,Niavaran,"6,800,000,000",226666.67,370146.67211
3263,105,2,True,True,True,Dorous,"5,600,000,000",186666.67,304826.67211


In [5]:
MEH = data.groupby(['Address'])['Price(USD)_2023'].sum().sort_values(ascending=False).head().reset_index()
MEH

,Address,Price(USD)_2023
0,Gheitarieh,7.705017e+07
1,Niavaran,7.243106e+07
2,Farmanieh,6.299488e+07
3,Saadat Abad,6.026641e+07
4,Shahrake Gharb,3.721188e+07


In [6]:
data.corr()

,Area,Room,Parking,Warehouse,Elevator,Price(USD),Price(USD)_2023
Area,1.000000,0.670137,0.199765,0.073080,0.047987,0.720990,0.720990
Room,0.670137,1.000000,0.279249,0.126244,0.183216,0.569476,0.569476
Parking,0.199765,0.279249,1.000000,0.428073,0.429527,0.192993,0.192993
Warehouse,0.073080,0.126244,0.428073,1.000000,0.204242,0.109675,0.109675
Elevator,0.047987,0.183216,0.429527,0.204242,1.000000,0.110417,0.110417
Price(USD),0.720990,0.569476,0.192993,0.109675,0.110417,1.000000,1.000000
Price(USD)_2023,0.720990,0.569476,0.192993,0.109675,0.110417,1.000000,1.000000


In [16]:
X = np.array((data["Area"] , data["Room"] , data["Parking"] , data["Elevator"])).T
Y = np.array(data[["Price(USD)_2023"]])

In [8]:
X_train , X_test , Y_train , Y_test = train_test_split(X,Y , test_size=0.2 , random_state=40)

In [9]:
# train
lls = LLS()
lls.fit(X_train , Y_train)
y_pred = lls.predict(X_test)

In [10]:
MAE_lls = lls.evaluate(X_test , Y_test , "mae")
MSE_lls = lls.evaluate(X_test , Y_test , "mse")
RMSE_lls = lls.evaluate(X_test , Y_test , "rmse")

print("LLS MAE =" , MAE_lls)
print("LLS MSE =" , MSE_lls)
print("LLS RMSE =" , RMSE_lls)

LLS MAE = 161371.087771871
LLS MSE = 92193498222.78737
LLS RMSE = 303633.82259357633


In [11]:
lls_sk = LinearRegression()
lls_sk.fit(X_train , Y_train)
y_pred_sk = lls_sk.predict(X_test)

In [12]:
MAE_linreg = mean_absolute_error(Y_test , y_pred_sk)
MSE_linreg = mean_squared_error(Y_test , y_pred_sk)
RMSE_linreg = np.sqrt(MSE_linreg)

print(MAE_linreg)
print(MSE_linreg)
print(RMSE_linreg)

152796.901678096
84232115103.3494
290227.69527277956


In [13]:
ridge = RidgeCV()
ridge.fit(X_train , Y_train)
Y_pred_ridge = ridge.predict(X_test)

In [14]:
MAE_ridge = mean_absolute_error(Y_test , Y_pred_ridge)
MSE_ridge = mean_squared_error(Y_test , Y_pred_ridge)
RMSE_ridge = np.sqrt(MSE_ridge)

print("RidgeCV MAE =" , MAE_ridge)
print("RidgeCV MSE =" , MSE_ridge)
print("RidgeCV RMSE =" , RMSE_ridge)

RidgeCV MAE = 152564.49227009708
RidgeCV MSE = 84201064451.13292
RidgeCV RMSE = 290174.1967355694
